# Pipeline Notebook

---
## Train a Classifier - SVM

In [ ]:
import numpy as np
import cv2
import pickle
import glob
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.svm import LinearSVC
from skimage.feature import hog
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
%matplotlib inline

### Load Training Data

In [ ]:
CAR_DIR_REGEX = '../dataset/vehicles/**/*.png'
NOCAR_DIR_REGEX = '../dataset/non-vehicles/**/*.png'

car_fnames = glob.glob(CAR_DIR_REGEX, recursive=True)
nocar_fnames = glob.glob(NOCAR_DIR_REGEX, recursive=True)

print('Loading data...')
print()
car_imgs = np.array([mpimg.imread(fname) for fname in car_fnames])
nocar_imgs = np.array([mpimg.imread(fname) for fname in nocar_fnames])

car_imgs_n = car_imgs.shape[0]
nocar_imgs_n = nocar_imgs.shape[0]

print('Loaded {} images of cars.'.format(car_imgs_n))
print('Loaded {} images of non-cars.'.format(nocar_imgs_n))
print()
print('Single image has size {}.'.format(car_imgs.shape[1:3]))
print('Single image has {} colors.'.format(car_imgs.shape[3]))

### Explore Training Data

In [ ]:
def display_images(images, title=None, limit=16, need_shuffle=False):
    if need_shuffle==True:
        images = shuffle(images, random_state=0)

    img_count = limit if (len(images)>limit) else len(images)
    m_cols = 8
    m_rows = int(img_count/m_cols) + (int(img_count/m_cols)==0)

    plt.figure(figsize=(15,4))
    for idx in range(limit):
        plt.subplot(m_rows, m_cols, idx+1)
        plt.axis('off')
        if title is not None: plt.title(title[idx])
        plt.imshow(images[idx])

    plt.show()
    return

print('Cars')
display_images(car_imgs, need_shuffle=True)
print('No-Cars')
display_images(nocar_imgs, need_shuffle=True)

In [ ]:
def display_hist_classes(cars_images, nocars_images):
    xticks = np.arange(4)
    ind = xticks[1:3]
    width = 0.65

    class_counts = (len(cars_images), len(nocars_images))
    plt.bar(ind, class_counts, width, facecolor='green', alpha=0.5, align='center')
    plt.xticks(xticks, ('', 'Cars', 'No-Cars', ''))
    plt.yticks(np.arange(0, int(1.2*max(class_counts)), 1000))
    plt.grid(True)
    plt.show()
    return

display_hist_classes(car_imgs, nocar_imgs)

### Define Training Features

In [ ]:
# Choose two samples randomly
titles = ['Car', 'No-Car']
car_img = car_imgs[np.random.randint(car_imgs_n)]
nocar_img = nocar_imgs[np.random.randint(nocar_imgs_n)]


display_images([car_img, nocar_img], title=titles, limit=2)

### Color Features

In [ ]:
def compute_color_hist(img, color_space='RGB', nbins=32, bins_range=(0, 256)):
    """ Computes the histograms of the given image on all of the 3 channels.
        Eventually, it converts the image to a new color space.
        Before computing the histogram, the pixel values are
        scaled to match the bins_range.
    """
    channels = img.shape[2]
    w, h = img.shape[1], car_img.shape[0]
    scaler = MinMaxScaler(feature_range=bins_range, copy=False)

    # color space convertion
    if color_space == 'HSV':
        conv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    elif color_space == 'HLS':
        conv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    elif color_space == 'LUV':
        conv_img = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    elif color_space == 'YUV':
        conv_img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    else:
        conv_img = np.copy(img)

    histograms = []
    for idx in range(channels):
        # get the channel idx and reshape
        # the array for the scaler
        ch = conv_img[:,:,idx].reshape(w*h,1)
        # scale the values to the given range
        ch_norm = scaler.fit_transform(ch)
        # compute the histogram
        ch_hist = np.histogram(ch_norm, bins=nbins, range=bins_range)
        # append the histogram
        histograms.append(ch_hist)

    return np.array(histograms)

def compare_color_hists(car_histograms, nocar_histograms, ch_names=['Red','Green','Blue']):
    """ Plots the histogram of each image channel in
        such a way to enhance Car and No-Car color
        channel features.
    """
    channels = car_histograms.shape[0]

    plt.figure(figsize=(15,4))
    for idx in range(channels):
        plt.subplot(1, 3, idx+1)
        if ch_names is not None: plt.title(ch_names[idx])

        # compute bar center using bin ranges
        x_ticks = car_histograms[idx][1][1:] - car_histograms[idx][1][1]/2
        car_data = plt.plot(x_ticks, car_histograms[idx][0], 'b', label='Car')
        nocar_data = plt.plot(x_ticks, nocar_histograms[idx][0], 'r', label='No-Car')

        plt.legend()
        plt.tight_layout()

    plt.show()
    return

In [ ]:
# RGB
rgb_car_hists = compute_color_hist(car_img)
rgb_nocar_hists = compute_color_hist(nocar_img)
compare_color_hists(rgb_car_hists, rgb_nocar_hists)

# HSV
hsv_car_hists = compute_color_hist(car_img, color_space='HSV')
hsv_nocar_hists = compute_color_hist(nocar_img, color_space='HSV')
compare_color_hists(hsv_car_hists, hsv_nocar_hists, ch_names=['Hue','Saturation','Value'])

# HSL
hls_car_hists = compute_color_hist(car_img, color_space='HLS')
hls_nocar_hists = compute_color_hist(nocar_img, color_space='HLS')
compare_color_hists(hls_car_hists, hls_nocar_hists, ch_names=['Hue','Lightness','Saturation'])

### Gradient Features

In [ ]:
def compute_hog(img, pix_per_cell=8, cell_per_block=2, orient=9, vis=False, feature_vec=True):
    """ Computes the Histogram of Oriented Gradient.
        The total number of features in your final feature vector
        will be the total number of block positions multiplied by
        the number of cells per block, times the number of orientations.
    
    img: a single color channel or grayscaled image.
    
    pix_per_cell: specifies the cell size over which each gradient
        histogram is computed. This paramater is passed as a 2-tuple
        but cells are commonly chosen to be square.

    cell_per_block: passed as a 2-tuple, and specifies the local area
        over which the histogram counts in a given cell will be normalized.
        Block normalization is not necessarily required, but generally
        leads to a more robust feature set.
    
    orient: specified as an integer, and represents the number of
        orientation bins that the gradient information will be split
        up into in the histogram. Typical values are between 6 and 12 bins.

    vis: if True, the function returns an image as 2nd parameter
    
    feature_vec: if True, returns the HOG for the image as 1D (flattened) array.
    """
    if vis == True:
        features, hog_img = hog(img,
                                orientations=orient,
                                pixels_per_cell=(pix_per_cell, pix_per_cell),
                                cells_per_block=(cell_per_block, cell_per_block),
                                transform_sqrt=False, 
                                visualise=True,
                                feature_vector=False)
        return features, hog_img
    else:      
        features = hog(img,
                       orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=False, 
                       visualise=False,
                       feature_vector=feature_vec)
        return features

def compare_hog_hists(car_histogram, nocar_histogram):
    plt.figure(figsize=(10,4))

    plt.subplot(1, 2, 1)
    plt.title("Car HOG")
    plt.imshow(car_histogram)

    plt.subplot(1, 2, 2)
    plt.title("No-Car HOG")
    plt.imshow(nocar_histogram)

    plt.show()
    return

In [ ]:
# From the color features analysis we have choosen HSV S-channel
car_hsv_img = cv2.cvtColor(car_img, cv2.COLOR_RGB2HSV)
nocar_hsv_img = cv2.cvtColor(nocar_img, cv2.COLOR_RGB2HSV)
car_s_ch = car_hsv_img[:,:,1]
nocar_s_ch = nocar_hsv_img[:,:,1]

# Compute HOG
car_features, car_hog_img = compute_hog(car_s_ch, vis=True, feature_vec=False)
nocar_features, nocar_hog_img = compute_hog(nocar_s_ch, vis=True, feature_vec=False)

# Display HOG
compare_hog_hists(car_hog_img, nocar_hog_img)

### Combine and Normalize Feature Vectors

In [ ]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32), hist_bins=32, hist_range=(0, 256)):

    # Create a list to append feature vectors to
    features = []

    # Iterate through the list of images
    for image in imgs:
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        else: feature_image = np.copy(image)      
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        # Compute HOG
        ch1_hog_hist = compute_hog(feature_image[:,:,0])
        ch2_hog_hist = compute_hog(feature_image[:,:,1])
        ch3_hog_hist = compute_hog(feature_image[:,:,2])
        hog_features = np.concatenate((ch1_hog_hist, ch2_hog_hist, ch3_hog_hist))
        # Append the new feature vector to the features list
        #features.append(np.concatenate((spatial_features, hist_features, hog_features)))
        features.append(np.concatenate((spatial_features, ch2_hog_hist)))
        #features.append(hog_features)
    # Return list of feature vectors
    return features

In [ ]:
car_features = extract_features(car_imgs,
                                      cspace='HSV',
                                      spatial_size=(32, 32),
                                      hist_bins=32,
                                      hist_range=(0, 256))
notcar_features = extract_features(nocar_imgs,
                                         cspace='HSV',
                                         spatial_size=(32, 32),
                                         hist_bins=32,
                                         hist_range=(0, 256))

In [ ]:
print(car_imgs[0].shape)
print(np.array(car_features).shape)

if len(car_features) > 0:
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)
    print(X.shape)
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
    car_ind = np.random.randint(0, len(car_imgs))
    nocar_ind = np.random.randint(0, len(nocar_imgs))
    # Plot an example of raw and scaled features
    fig = plt.figure(figsize=(12,4))

    plt.subplot(231)
    plt.imshow(car_imgs[car_ind])
    plt.title('Original Image')
    plt.subplot(232)
    plt.plot(X[car_ind])
    plt.title('Raw Features')
    plt.subplot(233)
    plt.plot(scaled_X[car_ind], 'g')
    plt.title('Normalized Features')
    
    plt.subplot(234)
    plt.imshow(nocar_imgs[nocar_ind])
    plt.title('Original Image')
    plt.subplot(235)
    plt.plot(X[len(car_imgs) + nocar_ind])
    plt.title('Raw Features')
    plt.subplot(236)
    plt.plot(scaled_X[len(car_imgs) + nocar_ind], 'g')
    plt.title('Normalized Features')
    fig.tight_layout()
else: 
    print('Your function only returns empty feature vectors...')

### Train Test Split

In [ ]:
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

print("Using {} samples for training".format(len(X_train)))
print("Using {} samples for testing".format(len(X_test)))

### Training a Linear SVM

In [ ]:
# Use a linear SVC 
svc = LinearSVC()

# Check the training time for the SVC
t0=time.time()
svc.fit(X_train, y_train)
t1 = time.time()

print(round(t1-t0, 2), 'Seconds to train SVC...')

### Evaluation

In [ ]:
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

In [ ]:
# Check the prediction time for a single sample
t0=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t1 = time.time()
print(round(t1-t0, 5), 'Seconds to predict', n_predict,'labels with SVC')

---
## Sliding Window Search

### Define Searching Area

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, tuple(bbox[0]), tuple(bbox[1]), color, thick)
    # Return the image copy with boxes drawn
    return imcopy

# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None],
                     xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]

    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]

    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))

    # Compute the number of windows in x/y
    nx_windows = np.int(xspan/nx_pix_per_step) - 1
    ny_windows = np.int(yspan/ny_pix_per_step) - 1

    # Initialize a list to append window positions to
    window_list = []

    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))

    # Return the list of windows
    return window_list

### Searching

In [ ]:
image = mpimg.imread('../test_images/test1.jpg')
w, h = image.shape[1], image.shape[0]
windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[np.int(h/2), None], 
                    xy_window=(128, 128), xy_overlap=(0.5, 0.5))
                       
print("Drawing {} windows".format(len(windows)))
print(windows[0])
window_img = draw_boxes(image, windows, color=(0, 0, 255), thick=6)                    
plt.imshow(window_img)
plt.show()

In [ ]:
def extract_sub_imgs(img, windows):
    sub_imgs = []

    for win in windows:
        xstart, xstop = win[0][0], win[1][0]
        ystart, ystop = win[0][1], win[1][1]
        sub_img = img[ystart:ystop,xstart:xstop,:]
        sub_img = cv2.resize(sub_img, (64,64))
        sub_imgs.append(sub_img)

    return np.array(sub_imgs)

# 1. extract sub-images
sub_imgs = extract_sub_imgs(image, windows)
print(sub_imgs.shape)

# 2. extract features
features = extract_features(sub_imgs, cspace='HSV', spatial_size=(32, 32), hist_bins=32, hist_range=(0, 256))

# 3. Normalize
X = np.array(features).astype(np.float64)
scaled_X = X_scaler.transform(X)

# 4. Classify
pred = svc.predict(scaled_X)

# 5. Display results
car_indexes = np.nonzero(pred)
car_boxes = np.asarray(windows)[car_indexes]
window_img = draw_boxes(image, car_boxes, color=(0, 255, 0), thick=6)                    
plt.imshow(window_img)
plt.show()

### Identifiy Cars Position - Centroid Method

---
## Pipeline - Video